In [15]:
using  Revise, Rasters, GeoFormatTypes, GeoTIFF, ArchGDAL
using Statistics

include("Julia//Parameters.jl")
include("Julia//GeoRaster.jl")
include("Julia//GeoNetcdf.jl")
include("Julia//PlotParameter.jl")
include("Julia//GeoPlot.jl")

Path_Home = raw"C:\OSGeo4W\Gis\FABDEM\IRELAND_MOSAIC0"
Path_Output = raw"C:\OSGeo4W\Gis\FABDEM\IRELAND_OUTPUT"
Files =readdir(Path_Home)


2-element Vector{String}:
 "N51W008_FABDEM_V1-0.tif"
 "N51W009_FABDEM_V1-0.tif"

In [21]:
Maps = []
for iiFile in Files
    Dem_Map₀ = Rasters.Raster(joinpath(Path_Home, iiFile))
    Dem_Map  = Rasters.replace_missing(Dem_Map₀, missingval=NaN)
    Maps     = push!(Maps, Dem_Map)
end


In [24]:
Mosaic = Rasters.mosaic(Statistics.mean, Maps; missingval=NaN, progress=true)


DimensionalData.Dimensions.Lookups.SelectorError{Projected{Float64, Vector{Float64}, DimensionalData.Dimensions.Lookups.ForwardOrdered, DimensionalData.Dimensions.Lookups.Regular{Float64}, DimensionalData.Dimensions.Lookups.Intervals{DimensionalData.Dimensions.Lookups.Center}, DimensionalData.Dimensions.Lookups.NoMetadata, WellKnownText{GeoFormatTypes.CRS}, Nothing, X{Colon}}, Float64}: SelectorError: attempt to select -7.9956249999999995 from lookup Projected{Float64, Vector{Float64}, DimensionalData.Dimensions.Lookups.ForwardOrdered, DimensionalData.Dimensions.Lookups.Regular{Float64}, DimensionalData.Dimensions.Lookups.Intervals{DimensionalData.Dimensions.Lookups.Center}, DimensionalData.Dimensions.Lookups.NoMetadata, WellKnownText{GeoFormatTypes.CRS}, Nothing, X{Colon}} with bounds (-8.000208333333333, -7.000208333333334)


In [5]:
Param_Crs_GeoFormat = GeoFormatTypes.convert(WellKnownText, EPSG(Param_Crs))

Mosaic_Resample = Rasters.resample(Mosaic; res=ΔX₂, method=ResampleMethod₂, crs=Param_Crs_GeoFormat)
#  Mosaic_Resample = Rasters.replace_missing(Mosaic_Resample, missingval=NaN)


┌ 21055×28024 Raster{Float64, 2} ┐
├────────────────────────────────┴─────────────────────────────────────── dims ┐
  ↓ X Projected{Float64} -10566.616338923981:20.0:410513.383661076 ForwardOrdered Regular Intervals{Start},
  → Y Projected{Float64} 532331.122395666:-20.0:-28128.877604334033 ReverseOrdered Regular Intervals{Start}
├──────────────────────────────────────────────────────────────────── metadata ┤
  Metadata{Rasters.GDALsource} of Dict{String, Any} with 1 entry:
  "filepath" => "/vsimem/tmp"
├────────────────────────────────────────────────────────────────────── raster ┤
  missingval: NaN
  extent: Extent(X = (-10566.616338923981, 410533.383661076), Y = (-28128.877604334033, 532351.122395666))
  crs: PROJCS["TM65 / Irish Grid",GEOGCS["TM65",DATUM["TM65",SPHEROID["Airy...
└──────────────────────────────────────────────────────────────────────────────┘
      ↓ →          5.32331e5    5.32311e5  …  -28088.9  -28108.9  -28128.9
 -10566.6        NaN          NaN                N

In [6]:
include("Julia//GeoRaster.jl")

# Getting the metadata
	Metadatas = geoRaster.RASTER_METADATA(Mosaic_Resample; Verbose=true)

	Longitude, Latitude = Rasters.X(Metadatas.Coord_X_Left: Metadatas.ΔX: Metadatas.Coord_X_Right, crs=Metadatas.Crs_GeoFormat), Rasters.Y(Metadatas.Coord_Y_Top: Metadatas.ΔY: Metadatas.Coord_Y_Bottom, crs=Metadatas.Crs_GeoFormat)

	Mosaic_Resample, Mosaic_Boarder₀ = geoRaster.DEM_CORRECT_BOARDERS!(; Dem=Mosaic_Resample, Latitude, Longitude, Crs=Metadatas.Crs_GeoFormat, iiParam_GaugeCoordinate=Param_GaugeCoordinate)

	Mosaic_Boarder = Rasters.replace_missing(Mosaic_Boarder₀, missingval=NaN)


Param_Crs = 29902
ΔX = 20.0
ΔY = -20.0
N_Width  = 21055
N_Height = 28024
Coord_X_Left = -10566.616338923981, Coord_X_Right = 410513.383661076
Coord_Y_Top = 532331.122395666, Coord_Y_Bottom = -28128.877604334033


UndefVarError: UndefVarError: `DEM_CORRECT_BOARDERS!` not defined in `Main.geoRaster`
Suggestion: check for spelling errors or missing imports.

In [7]:
	Path_OutputJulia_Dem = joinpath(Path_Output, "BOARDERS_Ireland_FABDEM.tiff")
	Rasters.write(Path_OutputJulia_Dem, Mosaic_Boarder; ext=".tiff", missingval= NaN, force=true, verbose=true)


UndefVarError: UndefVarError: `Mosaic_Boarder` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

In [8]:

# Writting the output
	Path_OutputJulia_Dem = joinpath(Path_Output, "DEM_Ireland_FABDEM2.tiff")
	Rasters.write(Path_OutputJulia_Dem, Mosaic_Resample; ext=".tiff", missingval= NaN, force=true, verbose=true)


"C:\\OSGeo4W\\Gis\\FABDEM\\IRELAND_OUTPUT\\DEM_Ireland_FABDEM2.tiff"

In [9]:
# geoPlot.HEATMAP(;🎏_Colorbar=true, Input= Mosaic, Title=string("iiFile"), Label="True [m]")
# using GLMakie
# fig = GLMakie.Figure()
# ga = Axis(
#     fig[1, 1] # any cell of the figure's layout
#     # dest = "+proj=wintri", # the CRS in which you want to plot
# )
# # lines!(Maps, GeoMakie.coastlines()) # plot coastlines from Natural Earth as a reference
# # # You can plot your data the same way you would in Makie
# GLMakie.heatmap!(ga, Mosaic)
# fig
